<a href="https://colab.research.google.com/github/agatagruza/private-ai/blob/master/SPAIC_Project13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 13: Securing Federated Learning

##Trusted Aggregator

In [0]:
pip install syft

In [28]:
import torch as th
import syft as sy
hook = sy.TorchHook(th)
from torch import nn, optim

W0728 05:34:43.783387 140672446506880 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0728 05:34:43.800626 140672446506880 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
# creatring virtual workers
bob = sy.VirtualWorker(hook, id = "bob") # data owner
alice = sy.VirtualWorker(hook, id = "alice") # data owner
# secure worker is a trusted 3rd party that is able to perform good aggregation
secure_worker = sy.VirtualWorker(hook, id = "secure_worker")

In [0]:
# Inrom worker, that other workers exsist. 
# Those are references to the other workers. 
bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([bob, alice])

In [0]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True) 
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True) 

In [0]:
# get pointers to training data on each worker by
# sending some trainig data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

In [0]:
alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [38]:
print(bobs_data)
print(bobs_target)
print(alices_data)
print(alices_target)

(Wrapper)>[PointerTensor | me:42471278022 -> bob:83437141932]
(Wrapper)>[PointerTensor | me:91637266804 -> bob:41690757274]
(Wrapper)>[PointerTensor | me:38554808975 -> alice:32962655949]
(Wrapper)>[PointerTensor | me:53281778580 -> alice:97519204886]


In [39]:
# Initialize a Toy Model
model = nn.Linear(2, 1)
print(model)

Linear(in_features=2, out_features=1, bias=True)


In [0]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

In [41]:
print(bobs_model)
print(alices_model)

Linear(in_features=2, out_features=1, bias=True)
Linear(in_features=2, out_features=1, bias=True)


In [0]:
# Two optimizers. One for the parameters Bob's model and one for the parametrs of alice's
bobs_optimizer = optim.SGD(params=bobs_model.parameters(), lr = 0.1)
alices_optimizer = optim.SGD(params=alices_model.parameters(), lr = 0.1)

In [43]:
print(bobs_optimizer)
print(alices_optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    momentum: 0
    nesterov: False
    weight_decay: 0
)
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [44]:
for round in range(10):

    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_optim = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_optim = optim.SGD(params=alices_model.parameters(), lr=0.1)

    for i in range(10):
        # Bobs model in train
        bobs_optim.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
        bobs_loss.backward()

        bobs_optim.step()
        bobs_loss = bobs_loss.get().data
        bobs_loss

        # Alices model in train
        alices_optim.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) ** 2).sum()
        alices_loss.backward()

        alices_optim.step()
        alices_loss = alices_loss.get().data  
        alices_loss
        
    # It iterates through every parameter in alices model and calls .move() 
    # on that parameter inline.
    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)

    # AVERAGING ALICE AND BOB TOGETEHR !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 
    with th.no_grad():
      model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
      model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
  
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0222) Alice:tensor(0.0054)
Bob:tensor(0.0077) Alice:tensor(0.0005)
Bob:tensor(0.0034) Alice:tensor(5.9335e-06)
Bob:tensor(0.0018) Alice:tensor(3.1658e-05)
Bob:tensor(0.0011) Alice:tensor(6.9191e-05)
Bob:tensor(0.0007) Alice:tensor(7.6473e-05)
Bob:tensor(0.0005) Alice:tensor(6.8336e-05)
Bob:tensor(0.0004) Alice:tensor(5.5857e-05)
Bob:tensor(0.0003) Alice:tensor(4.3798e-05)
Bob:tensor(0.0002) Alice:tensor(3.3649e-05)


In [45]:
secure_worker._objects

{941576072: Parameter containing:
 tensor([0.0131], requires_grad=True), 15671365667: Parameter containing:
 tensor([0.1700], requires_grad=True), 25516009287: Parameter containing:
 tensor([0.1278], requires_grad=True), 26027544051: Parameter containing:
 tensor([[0.8246, 0.0096]], requires_grad=True), 29318993004: Parameter containing:
 tensor([[0.7310, 0.0107]], requires_grad=True), 29990513963: Parameter containing:
 tensor([0.0690], requires_grad=True), 31296840173: Parameter containing:
 tensor([[ 0.7617, -0.0246]], requires_grad=True), 34657443041: Parameter containing:
 tensor([[0.7668, 0.0112]], requires_grad=True), 36149662432: Parameter containing:
 tensor([[ 0.8205, -0.0183]], requires_grad=True), 41678540588: Parameter containing:
 tensor([[0.6895, 0.0072]], requires_grad=True), 42923573251: Parameter containing:
 tensor([[0.8681, 0.0074]], requires_grad=True), 44050883456: Parameter containing:
 tensor([0.1172], requires_grad=True), 44229204257: Parameter containing:
 ten

In [46]:
secure_worker.clear_objects()

<VirtualWorker id:secure_worker #objects:0>

**And voila.We have Federated Learnign  where the gradient aggregation happens on a neutral third party.**

#Intro to Additive Secret Sharing

##Build Methods for Encrypted, Decrypted, and Add
Write a general method for enryption, decryption, andd add. Store shres for a variable in a tuple.</br>
**Encryption** is a bout henerating three shares such that when we sum then together and take a modulus of that sum,
it will decrypt to the correct value. 

In [0]:
import random

Q = 23740629843760239486723

def encrypt(x, n_share=3):
    
    shares = list()
    for i in range(n_share-1):
        shares.append(random.randint(0,Q))  
    shares.append(Q - (sum(shares) % Q) + x)
    return tuple(shares)

In [0]:
print(encrypt(x, n_share=5))

In [0]:
def decrypt(shares):
    return sum(shares) % Q # make sure It will not overflow !!!!!!!!!!!

In [6]:
decrypt(encrypt(5))

5

In [0]:
def add(a, b):
    c = list()
    assert(len(a) == len(b))
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q) 
    return tuple(c)

In [10]:
decrypt(add(encrypt(5), encrypt(10)))

15

In [11]:
x = encrypt(5)
x

(22344166337189812055967, 2252949701173554910847, 22884143649157112006637)

In [12]:
y = encrypt(10)
y

(2043202692803159314981, 19195557481184925635575, 2501869669772154536177)

In [13]:
z = add(x,y)
z

(646739186232731884225, 21448507182358480546422, 1645383475169027056091)

##Intro to Fixed Precision Encoding
Aggregate gradient using Secret Sharing Technique. Because weights are decimal (floating points numbers), use **fixing presission encoding**. 

In [0]:
BASE=10 # we are gonna use notrmal decoding. BASE 2 is another popular one
PRECISION=4 # 4 decimal places for storage
Q = 23740629843760239486723 # allow is to handle negative numbers

In [0]:
def encode(num):
    return int((num * (BASE ** PRECISION)) % Q)

def decode(num_fixed_prec):
    # It will take into account negative numbers
    return (num_fixed_prec if num_fixed_prec <= Q/2 else num_fixed_prec - Q) / BASE**PRECISION

In [18]:
encode(0.5)

5000

In [21]:
decode(5000)

0.5

In [23]:
encode(-0.5) # It will wrapped around other side of the queue

23740629843760239345664

##Secret Sharing Using PySyft

In [0]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [0]:
x = th.tensor([1,2,3,4,5])

In [57]:
x

tensor([1, 2, 3, 4, 5])

In [0]:
# Pointers to three different shares
x = x.share(bob, alice, secure_worker)

In [60]:
x

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:34868780781 -> bob:56026770914]
	-> (Wrapper)>[PointerTensor | me:78276092707 -> alice:76987362942]
	-> (Wrapper)>[PointerTensor | me:47576807611 -> secure_worker:64647945349]
	*crypto provider: me*

In [61]:
bob._objects

{56026770914: tensor([3872608970256795412, 3874236013703513915, 4588073900419218553,
          369704292436714266,  399986568358771403])}

In [62]:
alice._objects

{76987362942: tensor([-1604052243092631836, -2707457908598827083,  -552573919869168871,
          1289559945729362148,   249962563086881744])}

In [63]:
y = x + x
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:65965424924 -> bob:47045607957]
	-> (Wrapper)>[PointerTensor | me:14643852280 -> alice:11868830632]
	-> (Wrapper)>[PointerTensor | me:21628621268 -> secure_worker:82960399997]
	*crypto provider: me*

In [64]:
y.get()

tensor([ 2,  4,  6,  8, 10])

##Fixed Precision using PySyft

In [65]:
x = th.tensor([0.1,0.2,0.3])
x

tensor([0.1000, 0.2000, 0.3000])

In [0]:
x = x.fix_prec()

In [67]:
x

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300])

In [69]:
y = x + x
y

(Wrapper)>FixedPrecisionTensor>tensor([200, 400, 600])

In [68]:
type(x)

syft.frameworks.torch.tensors.interpreters.native.Tensor

In [70]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

##Shared Fixed Precision

In [0]:
x = th.tensor([0.1, 0.2, 0.3])
x = x.fix_prec().share(bob, alice, secure_worker)

In [72]:
x

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:1056585884 -> bob:29027314579]
	-> (Wrapper)>[PointerTensor | me:43592471694 -> alice:88290247163]
	-> (Wrapper)>[PointerTensor | me:192075606 -> secure_worker:52409415714]
	*crypto provider: me*

In [73]:
y = x + x
y

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:60370084891 -> bob:63252406961]
	-> (Wrapper)>[PointerTensor | me:71037938492 -> alice:29960196020]
	-> (Wrapper)>[PointerTensor | me:63308745653 -> secure_worker:73163716851]
	*crypto provider: me*

In [74]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])